# Approach

RAW --> zip
BRONZE --> csv sin fraudulent
SILVER --> parquet con las columnas seleccionadas y schema validado
GOLD --> silver pero con las siguientes columnas adicionales obtenidas usando LLM
   1. Responsabilidades (array de conceptos)
   2. Soft skills (array de conceptos)
   3. Hard skills (array de conceptos)

Salario mínimo y máximo
Experiencia requerida de las 3 ofertas con más salario
Top 5 industrias con más ofertas
Resumir requirements + description y extraer: --> GOLD 
   1. Responsabilidades (array de conceptos)
   2. Soft skills (array de conceptos)
   3. Hard skills (array de conceptos)

## Separate dataset (variables vs variables+predicted)

In [16]:
import pandas as pd
from ydata_profiling import ProfileReport
import pandera as pa

In [37]:
df = pd.read_csv('./s3/raw/real-or-fake-fake-jobposting-prediction.zip')

In [38]:
# get all columns but fraudulent
df = df.drop(columns=['fraudulent'])

In [39]:
df

,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function
0,1,Marketing Intern,"US, NY, New York",Marketing,NaN,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,NaN,0,1,0,Other,Internship,NaN,NaN,Marketing
1,2,Customer Service - Cloud Video Production,"NZ, , Auckland",Success,NaN,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,NaN,Marketing and Advertising,Customer Service
2,3,Commissioning Machinery Assistant (CMA),"US, IA, Wever",NaN,NaN,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,NaN,0,1,0,NaN,NaN,NaN,NaN,NaN
3,4,Account Executive - Washington DC,"US, DC, Washington",Sales,NaN,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales
4,5,Bill Review Manager,"US, FL, Fort Worth",NaN,NaN,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17875,17876,Account Director - Distribution,"CA, ON, Toronto",Sales,NaN,Vend is looking for some awesome new talent to...,Just in case this is the first time you’ve vis...,To ace this role you:Will eat comprehensive St...,What can you expect from us?We have an open cu...,0,1,1,Full-time,Mid-Senior level,NaN,Computer Software,Sales
17876,17877,Payroll Accountant,"US, PA, Philadelphia",Accounting,NaN,WebLinc is the e-commerce platform and service...,The Payroll Accountant will focus primarily on...,- B.A. or B.S. in Accounting- Desire to have f...,Health &amp; WellnessMedical planPrescription ...,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Internet,Accounting/Auditing
17877,17878,Project Cost Control Staff Engineer - Cost Con...,"US, TX, Houston",NaN,NaN,We Provide Full Time Permanent Positions for m...,Experienced Project Cost Control Staff Enginee...,At least 12 years professional experience.Abil...,NaN,0,0,0,Full-time,NaN,NaN,NaN,NaN
17878,17879,Graphic Designer,"NG, LA, Lagos",NaN,NaN,NaN,Nemsia Studios is looking for an experienced v...,1. Must be fluent in the latest versions of Co...,Competitive salary (compensation will be based...,0,0,1,Contract,Not Applicable,Professional,Graphic Design,Design


In [40]:
df.to_csv('./s3/bronze/real-or-fake-fake-jobposting-prediction.csv', index=False)

In [41]:
raw_report = ProfileReport(df, title='Real or Fake Fake Jobposting Prediction Report', explorative=True)
raw_report.to_file('./s3/bronze/real-or-fake-fake-jobposting-prediction-report.html')

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 168.04it/s]


# SILVER

In [42]:
import numpy as np

In [ ]:
# select columns job_id, title, salary_range, industry, location, description, requirements, required_experience
silver_df = df[['job_id', 'title', 'salary_range', 'industry', 'location', 'description', 'requirements', 'required_experience']]

In [62]:
# create two new columns lower_bound_salary and upper_bound_salary from salary_range using pandas and panderas for schema validation
splitted_df = (
    silver_df
    .fillna({'salary_range': '0-0'})
    .assign(
        lower_bound_salary=lambda x: x['salary_range'].str.split('-').str[0].astype(str),
        upper_bound_salary=lambda x: x['salary_range'].str.split('-').str[1].astype(str)
    ) 
)

# some lower_bound_salary and upper_bound_salary are not numbers, they have values like 'Oct' and 'Sep', ignore those rows
silver_df = (
    splitted_df
    .loc[lambda x: x['lower_bound_salary'].str.isnumeric()]
    .loc[lambda x: x['upper_bound_salary'].str.isnumeric()]
    .astype({
        'lower_bound_salary': float,
        'upper_bound_salary': float
    })
)

silver_df

,job_id,title,salary_range,industry,location,description,requirements,required_experience,lower_bound_salary,upper_bound_salary
0,1,Marketing Intern,0-0,NaN,"US, NY, New York","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,Internship,0.0,0.0
1,2,Customer Service - Cloud Video Production,0-0,Marketing and Advertising,"NZ, , Auckland",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,Not Applicable,0.0,0.0
2,3,Commissioning Machinery Assistant (CMA),0-0,NaN,"US, IA, Wever","Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,NaN,0.0,0.0
3,4,Account Executive - Washington DC,0-0,Computer Software,"US, DC, Washington",THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Mid-Senior level,0.0,0.0
4,5,Bill Review Manager,0-0,Hospital & Health Care,"US, FL, Fort Worth",JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Mid-Senior level,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
17875,17876,Account Director - Distribution,0-0,Computer Software,"CA, ON, Toronto",Just in case this is the first time you’ve vis...,To ace this role you:Will eat comprehensive St...,Mid-Senior level,0.0,0.0
17876,17877,Payroll Accountant,0-0,Internet,"US, PA, Philadelphia",The Payroll Accountant will focus primarily on...,- B.A. or B.S. in Accounting- Desire to have f...,Mid-Senior level,0.0,0.0
17877,17878,Project Cost Control Staff Engineer - Cost Con...,0-0,NaN,"US, TX, Houston",Experienced Project Cost Control Staff Enginee...,At least 12 years professional experience.Abil...,NaN,0.0,0.0
17878,17879,Graphic Designer,0-0,Graphic Design,"NG, LA, Lagos",Nemsia Studios is looking for an experienced v...,1. Must be fluent in the latest versions of Co...,Not Applicable,0.0,0.0


## Schema validation

In [95]:
silver_schema = pa.DataFrameSchema({
    'job_id': pa.Column(pa.Int, nullable=False),
    'title': pa.Column(pa.String, nullable=False),
    'salary_range': pa.Column(pa.String, nullable=True),
    'industry': pa.Column(pa.String, nullable=True),
    'location': pa.Column(pa.String, nullable=True),
    'description': pa.Column(pa.String, nullable=True),
    'requirements': pa.Column(pa.String, nullable=True),
    'required_experience': pa.Column(pa.String, nullable=True),
    'lower_bound_salary': pa.Column(pa.Float, checks=[
        pa.Check.greater_than_or_equal_to(0),
        pa.Check.less_than(1_000_000_000)
    ], nullable=False),
    'upper_bound_salary': pa.Column(pa.Float, nullable=False),
})

try:
    silver_schema.validate(silver_df)
except pa.errors.SchemaError as e:
    print("Schema errors and failure cases")
    df_error_cases = e.failure_cases
    print("\nDataFrame object that failed validation")
    df_error_data = e.data

In [83]:
silver_df.iloc[12251]

job_id                                                             12272
title                                              Technicians Required 
salary_range                                                         0-0
industry                                                             NaN
location                                                          NG, , 
description            Harbour Human Capital Solutions Limited on beh...
requirements           Successful applicants must possess the followi...
required_experience                                                  NaN
lower_bound_salary                                                   0.0
upper_bound_salary                                                   0.0
Name: 12271, dtype: object